In [92]:
#!/usr/bin/env python

import numpy as np
import matplotlib.pyplot as plt
import cv2

from scipy.spatial.transform import Rotation 

import rospy
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image, CameraInfo
from geometry_msgs.msg import Pose

# generate grid

# Data hardcoded

In [2]:
# For design purposes, to be replaced from acquisiton
marker1_pos = np.array([0.1, 0.66, 0.1])
marker1_rot = 0.78
gridsizex = 0.4
gridsizey = 0.5
rot_mat = np.array([[np.cos(marker1_rot), -np.sin(marker1_rot), 0],
                    [np.sin(marker1_rot), np.cos(marker1_rot), 0],
                    [0, 0, 1]])
marker2_pos = marker1_pos + rot_mat @ np.array([gridsizex, gridsizey, 0])

marker2_rot = marker1_rot

# Grid is an array of dictionnaries
markers = {"0": {"pos": marker1_pos, "rot": marker1_rot},
           "1": {"pos": marker2_pos, "rot": marker2_rot}}

# data from ROS

In [309]:
rospy.init_node("python_node_aruco",anonymous=True)
show=True
debug=False

In [324]:
#TODO : add a function to get the position of the marker in the camera frame

markers = {}

try:
    marker29 = rospy.wait_for_message("/aruco_simple/pose", Pose, timeout=0.1)
    marker29_pos = np.array([marker29.position.x, marker29.position.y, marker29.position.z])
    marker29_rotation = Rotation.from_quat([marker29.orientation.x, marker29.orientation.y,
                                            marker29.orientation.z, marker29.orientation.w])
    marker26 = rospy.wait_for_message("/aruco_simple/pose2", Pose, timeout=0.1)
    marker26_pos = np.array([marker26.position.x, marker26.position.y, marker26.position.z])
    marker26_rotation = Rotation.from_quat([marker26.orientation.x, marker26.orientation.y,
                                            marker26.orientation.z, marker26.orientation.w])
    markers = {"marker26": {"pos": marker26_pos, "rot": marker26_rotation.as_euler('XYZ', degrees=True)},
               "marker29": {"pos": marker29_pos, "rot": marker29_rotation.as_euler('XYZ', degrees=True)}}
    rot_mat = (marker26_rotation.as_matrix() + marker29_rotation.as_matrix()) / 2
except:
    rospy.logerr("Marker 26 or 29 not found")

if show:
    # Get image
    bridge = CvBridge()
    image_msg = rospy.wait_for_message("/aruco_simple/result", Image, timeout=0.1)
    image = bridge.imgmsg_to_cv2(image_msg, desired_encoding="passthrough")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow("image", image)
    cv2.waitKey(25)
# cv2.destroyAllWindows()
   
#TODO demain: Change le module ros pour capter plusieurs aruco
# S'abonner au topic du aruco detector
# reprojetter les points sur le plan de la table

In [325]:
#Number of cells
CNx = 6
CNy = 3.5
CNz = int(CNx - 1 + CNy - 1)

marker_TopLeft = "marker26"
marker_BottomRight = "marker29"
grid_rot= (markers[marker_BottomRight]["rot"] + markers[marker_TopLeft]["rot"])/2*np.pi/180
marker_vect = markers[marker_BottomRight]["pos"] - markers[marker_TopLeft]["pos"]

#grid size
GS = rot_mat.T @ marker_vect
grid = np.empty((int(CNx), int(CNy)), dtype=dict)

grid[0, 0] = {"pos": markers[marker_TopLeft]["pos"], "empty": True}
grid_data = {"pos": markers[marker_TopLeft]["pos"], "rot": grid_rot, "CNx": CNx, "CNy": CNy, "CNz": CNz}

In [326]:
#Change of basis and rotation
grid_offsetx = rot_mat @ np.array([GS[0]/(CNx - 1), 0, 0])
grid_offsety = rot_mat @ np.array([0, GS[1]/(CNy - 1), 0])
grid_offsetz = rot_mat @ np.array([0, 0, GS[2]/(7)])

for x in range(int(CNx)):
    for y in range(int(CNy)):
        if x%2 is not 0:
            offset = 0.5
        else:
            empty_flag = True
            offset = 0
        grid[x, y] = {"pos": grid[0, 0]["pos"] + (x)*grid_offsetx + (y + offset)*grid_offsety + (x+y)*grid_offsetz, "empty": empty_flag}

grid[0,0]["empty"] = False
grid[-1, -1]["empty"] = False

In [327]:
if debug:
    plt.figure()
    for marker_key in markers.keys():
        marker = markers[marker_key]
        print(marker["pos"])
        plt.scatter(marker["pos"][0], marker["pos"][1])
        plt.text(marker["pos"][0], marker["pos"][1], marker_key)
        plt.legend(["marker 0", "marker 1"])
    plt.scatter(grid_data["pos"][0], grid_data["pos"][1])
    plt.legend(["marker 0", "marker 1", "grid"])
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i, j]["empty"]:
                plt.scatter(grid[i, j]["pos"][0], grid[i, j]["pos"][1], marker="x", color="blue")
            else:
                plt.scatter(grid[i, j]["pos"][0], grid[i, j]["pos"][1], marker="x", color="black")
    plt.xlabel("x")
    plt.ylabel("y")
plt.show()

In [330]:
if show:
    camera_info = rospy.wait_for_message("/camera/color/camera_info", CameraInfo, timeout=0.1)
    intrinsic = np.array(camera_info.K).reshape((3, 3))
    
    grid_pxl = np.empty((grid.shape[0], grid.shape[1], 2))
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            xyw = intrinsic @ np.array(grid[i, j]["pos"])
            grid_pxl[i, j] = xyw[:2]/xyw[2]
            grid_pxl[i, j] = grid_pxl[i, j].astype(int)
    # Plot grid on image
    image_plt = image.copy()
    for i in range(grid_pxl.shape[0]):
        for j in range(grid_pxl.shape[1]):
            if grid[i, j]["empty"]:
                cv2.circle(image_plt, (int(grid_pxl[i, j][0]), int(grid_pxl[i, j][1])), 5, (0, 0, 255), -1)
            else:
                cv2.circle(image_plt, (int(grid_pxl[i, j][0]), int(grid_pxl[i, j][1])), 5, (0, 255, 0), -1)

    marker26_pxl = intrinsic @ np.array(markers["marker26"]["pos"])
    marker26_pxl/=marker26_pxl[2]

    marker29_pxl = intrinsic @ np.array(markers["marker29"]["pos"])
    marker29_pxl/=marker29_pxl[2]
    cv2.circle(image_plt, (int(marker26_pxl[0]), int(marker26_pxl[1])), 6, (255, 0, 0), 2)

    cv2.circle(image_plt, (int(marker29_pxl[0]), int(marker29_pxl[1])), 6, (255, 0, 0), 2)

    cv2.imshow("image_plt", image_plt)
    cv2.waitKey(25)